In [ ]:
# keep the biggest connected component and specify non-detected images (with no connected component detected)
import cv2
import os
import numpy as np

prediction_folder= '/path/to/dataset/predictions'

def keep_biggest_connected_component(image_path, output_path):
    # Get a list of all image files in the predictions path
    image_files = [f for f in os.listdir(image_path) if f.endswith(('.bmp'))]

    if not image_files:
        print("No image files found in the specified path.")
        return
    
    filenames_no_component = []
    no_filenames_no_component=0
    
    for image_file in image_files:
        # Read the image
        image = cv2.imread(os.path.join(image_path, image_file), cv2.IMREAD_GRAYSCALE)

        # Threshold the image to get a binary representation
        _, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

        # Find connected components
        _, labels, stats, _ = cv2.connectedComponentsWithStats(binary_image)

        # Check if any connected components were found
        if len(stats) <= 1:
            #print(f"No connected components found in {image_file}. keep the original file.")
            output_file = os.path.join(output_path,f"{image_file}")
            cv2.imwrite(output_file, image)
            filenames_no_component.append(image_file)
            no_filenames_no_component+=1
            continue

        # Find the index of the largest connected component
        largest_component_index = np.argmax(stats[1:, cv2.CC_STAT_AREA]) + 1

        # Create a mask to keep only the largest connected component
        mask = np.zeros_like(binary_image)
        mask[labels == largest_component_index] = 255

        # Save or process the result as needed
        output_file = os.path.join(output_path, f"{image_file}")
        cv2.imwrite(output_file, mask)
    
    if filenames_no_component:
        print(f"The filenames with no connected components are: {filenames_no_component}")
        
    else:
        print("No such files found.")

    return filenames_no_component, no_filenames_no_component


Predictions_bcc_path =  '/path/to/dataset/Result_att/Predictions_bcc'

# Ensure the output directory exists
os.makedirs(Predictions_bcc_path, exist_ok=True)

[f_n_c,no_filenames_no_cc]=keep_biggest_connected_component(prediction_folder, Predictions_bcc_path)
print(no_filenames_no_cc,' non-detected images (with no connected component detected) files')

In [ ]:
##keep the second biggest connected component for patients with a cyst larger than the RAZ

import os
import cv2
import numpy as np
from skimage.measure import label
from skimage.io import imsave

# Set folder path
prediction_folder= '/path/to/dataset/predictions'
output_folder = '/path/to/dataset/predictions_2bcc'
os.makedirs(output_folder, exist_ok=True)

# Function to process image, i.e. keep the second biggest connected component
def process_image(image_path, output_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Threshold to binary (you can adjust threshold value)
    _, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

    # Convert to boolean
    binary = binary > 0

    # Label connected components
    labeled = label(binary)

    # Get sizes of components
    component_sizes = np.bincount(labeled.ravel())
    component_sizes[0] = 0  # background is label 0

    if len(component_sizes) < 2:
        print(f"No second largest component found in {image_path}")
        return

    # Get label of second largest component
    second_largest_label = component_sizes.argsort()[-2]

    # Keep only second largest component
    second_largest = (labeled == second_largest_label).astype(np.uint8) * 255

    imsave(output_path, second_largest)

# Loop through all predictions
for filename in os.listdir(prediction_folder):
    if filename.lower().endswith(".bmp"):
        full_path = os.path.join(prediction_folder, filename)
        out_path = os.path.join(output_folder, filename)
        process_image(full_path, out_path)
        print(f"Processed: {filename}")


In [ ]:
# Fill any holes inside a binary object
import os
import cv2
import numpy as np

def fill_holes_inside_object(binary_object):
    # Create a kernel for morphological operations
    kernel = np.ones((15, 15), np.uint8)

    # Perform closing operation to fill holes inside the binary object
    closed_object = cv2.morphologyEx(binary_object, cv2.MORPH_CLOSE, kernel)

    return closed_object

def process_images(path1, path2):
    # Create output directory if it doesn't exist
    os.makedirs(path2, exist_ok=True)

    No_post_pro=0
    # List all files in the input directory
    files = [f for f in os.listdir(path1) if os.path.isfile(os.path.join(path1, f))]

    for file in files:
        # Read the binary image
        image_path = os.path.join(path1, file)
        binary_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Ensure the image is binary (convert to grayscale and threshold if necessary)
        _, binary_image = cv2.threshold(binary_image, 128, 255, cv2.THRESH_BINARY)

        # Assume you have a binary object and you want to fill holes inside it
        # Extract the region of interest (ROI) containing the object
        # You may need to perform segmentation or contour detection to isolate the object
        
        binary_object = binary_image.copy()

        # Find the largest connected component
        filled_object = fill_holes_inside_object(binary_object)
        output_path = os.path.join(path2, file)
        cv2.imwrite(output_path, filled_object)
        No_post_pro+=1
        
    print("No_post_pro",No_post_pro)

if __name__ == "__main__":
    path1 = '/path/to/dataset/Result_att/Predictions_bcc'
    path2 = path1 + '_Fill_holles'
    process_images(path1, path2)

In [ ]:
# calculate average of some images needed to be replaced (with no connected component detected)
from skimage import io
#import cv2
#import numpy as np
#import os

def read_binary_images(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".png") or filename.endswith(".bmp"):
            image_path = os.path.join(folder_path, filename)
            # Read image
            image = io.imread(image_path)
            images.append(image)
    return images

def average_binary_images(image_list):
    # Convert the list of images into numpy array
    images_array = np.array(image_list)
    # Sum the array along the first axis (image index)
    summed_array = np.sum(images_array, axis=0)
    # Divide by the number of images to get the average
    average_array = summed_array / len(image_list)
    return average_array

folder_path = r'/path/to/dataset/predictions_needs_averaging'
binary_images = read_binary_images(folder_path)
average_image = average_binary_images(binary_images)

# Save the average image
output_path =os.path.join(folder_path, "average_image.bmp")
io.imsave(output_path, average_image.astype(np.uint8))  # Ensure the data type is uint8 for binary image

print("Average image saved to:", output_path)

# save the result of average image into path and convert to binary

# Assuming your uint8 array is stored in a variable named 'uint8_array'
image = cv2.imread(output_path, cv2.IMREAD_GRAYSCALE)  

# Convert the image to a uint8 array
uint8_array = image.astype(np.uint8)

# Reshape the uint8 array into an image shape (height, width)
image_shape = (256, 256)  # specify the height and width of your image
binary_image = uint8_array.reshape(image_shape)

# Threshold the image to create a binary image
threshold_value = 127  # You can adjust this threshold value as per your requirement
_, binary_image = cv2.threshold(binary_image, threshold_value, 255, cv2.THRESH_BINARY)

# Show the binary image
output_path =os.path.join(folder_path, "average_image_b.bmp")
cv2.imwrite(output_path, binary_image)

print("Image saved successfully at:", output_path)

In [ ]:
## Show overlay of images, masks and predictions
#import cv2
#import numpy as np
#import os
import matplotlib.pyplot as plt

def plot_image_with_contours(image, mask, prediction):
    """
    Plots an image with contours of the mask and prediction.

    Args:
        image (numpy.ndarray): Original image (H x W x C).
        mask (numpy.ndarray): Ground-truth mask (H x W).
        prediction (numpy.ndarray): Predicted mask (H x W).
    """
    # Convert image to RGB if it is grayscale
    if len(image.shape) == 2:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    else:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Resize mask and prediction to match the image size
    mask_resized = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)
    prediction_resized = cv2.resize(prediction, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

    # Find contours for the resized mask and prediction
    mask_contours, _ = cv2.findContours(mask_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    prediction_contours, _ = cv2.findContours(prediction_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw contours on a copy of the original image
    image_with_contours = image_rgb.copy()
    cv2.drawContours(image_with_contours, mask_contours, -1, (0, 0, 255), 2)        # Blue for mask
    cv2.drawContours(image_with_contours, prediction_contours, -1, (255, 0, 0), 2)  # Red for prediction

    # Plot the image
    plt.figure(figsize=(3, 3))
    plt.imshow(image_with_contours)
    plt.axis("off")
    plt.title(f"{base_name}: Mask (Blue) and Prediction (Red)")

    # Save each figure with an incrementing filename
    filename = os.path.join(output_dir, f"{base_name}.png")
    plt.savefig(filename, bbox_inches='tight', dpi=300)
    
    plt.show()
    plt.close()

# Process folder of images
if __name__ == "__main__":
      
    main_path='/path/to/dataset/Fold1'
    image_folder = main_path+r'\Test\Images'
    mask_folder = main_path+r'\Test\Masks'
    prediction_folder = main_path+r'\Result_att\Predictions_PostPro'
    output_dir= main_path+r'\Result_att\Results-overlay'
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Ensure folders exist
    if not os.path.exists(image_folder) or not os.path.exists(mask_folder) or not os.path.exists(prediction_folder):
        print("Error: One or more folders do not exist.")
        exit()

    # List all files in the folders
    image_files = sorted(os.listdir(image_folder))
    mask_files = sorted(os.listdir(mask_folder))
    prediction_files = sorted(os.listdir(prediction_folder))

    # Process files that match by the first 11 characters to ensure working on the slices of the same patient
    for image_file in image_files:
        base_name = image_file[:11]  # Match based on first 11 characters
        matching_mask = next((f for f in mask_files if f.startswith(base_name)), None)
        matching_prediction = next((f for f in prediction_files if f.startswith(base_name)), None)

        if matching_mask and matching_prediction:
            # Load the files
            image_path = os.path.join(image_folder, image_file)
            mask_path = os.path.join(mask_folder, matching_mask)
            prediction_path = os.path.join(prediction_folder, matching_prediction)

            image = cv2.imread(image_path)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            prediction = cv2.imread(prediction_path, cv2.IMREAD_GRAYSCALE)

            # Check if files are loaded correctly
            if image is None:
                print(f"Error: Unable to load image from '{image_path}'")
                continue
            if mask is None:
                print(f"Error: Unable to load mask from '{mask_path}'")
                continue
            if prediction is None:
                print(f"Error: Unable to load prediction from '{prediction_path}'")
                continue

            # Ensure masks are binary (0 or 255)
            _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
            _, prediction = cv2.threshold(prediction, 127, 255, cv2.THRESH_BINARY)

            # Plot the image with contours
            plot_image_with_contours(image, mask, prediction)
        else:
            print(f"No matching mask or prediction for image '{image_file}'")